In [ ]:
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Charger un modèle pré-entraîné (par ex. VGG16)
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# 2. Préparer les images avec ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. / 255)

# Indiquer le répertoire des images
train_data = datagen.flow_from_directory(
    './dataset/car_or_truck/train',
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary',  # 'categorical' pour plusieurs classes
    shuffle=False
)

# 3. Extraire les caractéristiques avec VGG16
def extract_features(model, data):
    features = model.predict(data)
    return features.reshape(features.shape[0], -1)  # Aplatir les caractéristiques

# Extraire les caractéristiques des images
features = extract_features(vgg_model, train_data)

# Obtenir les étiquettes
labels = train_data.classes

# 4. Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 5. Entraîner XGBoost pour la classification en utilisant le GPU
xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    eval_metric='logloss'
)

xgb_model.fit(X_train, y_train)

# 6. Évaluer le modèle sur l'ensemble de test
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitude du modèle XGBoost: {accuracy:.2f}")

Found 5117 images belonging to 2 classes.


c:\Users\Skayne\Desktop\cours_tech\Machine learning\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 639s 16s/step
Exactitude du modèle XGBoost: 0.89


In [13]:
import numpy as np
from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow.keras.applications import VGG19  # Remplacer VGG16 par VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Charger un modèle pré-entraîné (par ex. VGG19)
vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# 2. Préparer les images avec ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. / 255)

# Indiquer le répertoire des images
train_data = datagen.flow_from_directory(
    './dataset/car_or_truck/train',
    target_size=(128, 128),
    batch_size=128,
    class_mode='binary',  # 'categorical' pour plusieurs classes
    shuffle=False
)

# 3. Extraire les caractéristiques avec VGG19
def extract_features(model, data):
    # Extraire les caractéristiques en prédiction
    features = model.predict(data)
    return features.reshape(features.shape[0], -1)  # Aplatir les caractéristiques pour XGBoost

# Extraire les caractéristiques des images d'entraînement
features = extract_features(vgg_model, train_data)

# Obtenir les étiquettes
labels = train_data.classes

# 4. Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 5. Entraîner XGBoost pour la classification en utilisant le GPU
xgb_model = LGBMClassifier(
    n_estimators=1000,
    eval_metric='logloss'
)

xgb_model.fit(X_train, y_train)

# 6. Évaluer le modèle sur l'ensemble de test
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitude du modèle XGBoost: {accuracy:.2f}")


Found 5117 images belonging to 2 classes.
40/40 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step
Exactitude du modèle XGBoost: 0.87


In [ ]:
import torch

# Vérifier si CUDA est disponible
print("CUDA Available: ", torch.cuda.is_available())

# Si CUDA est disponible, afficher le nom de ton GPU
if torch.cuda.is_available():
    print("GPU Name: ", torch.cuda.get_device_name(0))


CUDA Available:  True
GPU Name:  NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision import datasets, models
from sklearn.metrics import accuracy_score

# Vérifier si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Charger le modèle pré-entraîné ResNet
model = models.resnet18(pretrained=True)
model = model.to(device)  # Déplacer le modèle vers le GPU si disponible

# Appliquer les transformations sur les images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Charger les données d'entraînement
train_data = datasets.ImageFolder('./dataset/car_or_truck/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# Charger les données de test
test_data = datasets.ImageFolder('./dataset/car_or_truck/valid', transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

# Critère de perte et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle
model.train()
for epoch in range(10):  # 10 époques d'exemple
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss/len(train_loader):.4f}")

# Modèle en mode évaluation
model.eval()

# Initialisation des listes pour stocker les prédictions et les étiquettes réelles
all_preds = []
all_labels = []

# Parcourir les données de test
with torch.no_grad():  # Désactive le calcul des gradients pendant l'évaluation
    for inputs, labels in test_loader:
        # Transférer les données vers le GPU si disponible
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Prédictions avec le modèle
        outputs = model(inputs)
        
        # La classe prédite est celle avec la probabilité la plus élevée
        _, preds = torch.max(outputs, 1)
        
        # Ajouter les prédictions et les étiquettes réelles à la liste
        all_preds.extend(preds.cpu().numpy())  # Transférer sur le CPU pour l'utilisation avec numpy
        all_labels.extend(labels.cpu().numpy())

# Calculer l'exactitude
accuracy = accuracy_score(all_labels, all_preds)
print(f"Exactitude du modèle: {accuracy * 100:.2f}%")

Using device: cuda
Epoch [1/10], Loss: 0.5008
Epoch [2/10], Loss: 0.1253
Epoch [3/10], Loss: 0.1049
Epoch [4/10], Loss: 0.0806
Epoch [5/10], Loss: 0.0635
Epoch [6/10], Loss: 0.0655
Epoch [7/10], Loss: 0.0394
Epoch [8/10], Loss: 0.0392
Epoch [9/10], Loss: 0.0301
Epoch [10/10], Loss: 0.0454
Exactitude du modèle: 94.60%
